In [98]:
from osgeo import gdal
from osgeo import ogr
import os, sys
import numpy as np
import json

In [99]:
def extract_coord(output_poly): #fonction pour extraire les coordonnées géographiques à partir d'un fichier json
    
    extract_json = json.loads(output_poly)
    coordinates = extract_json['coordinates']
    xmin = coordinates[0][0][0][0]
    xmax = coordinates[0][0][1][0]
    ymin = coordinates[0][0][0][1]
    ymax = coordinates[0][0][2][1]
    
    return xmin,ymin,xmax,ymax

In [100]:
#code from https://github.com/tgrippa/Partimap/blob/cd72ecd2f3e32794eb3e0ee70399de1811b5399e/Notebooks/Data_preprocessing.ipynb
def normalise_01(image_data):
    image_data -= np.min(stack, axis=0, dtype=np.float32)
    image_data /= (np.max(stack, axis=0, dtype=np.float32) - np.min(stack, axis=0, dtype=np.float32))
    return image_data

In [110]:
rast = gdal.Open("Images/S1/s1_vv_small.tif") #ouverture du raster

vector = ogr.Open('Images/smaller/sample/sample_grid.gpkg') #ouverture de la fichier gkpg

layer = vector.GetLayer() #extraction des couches dans le gkpg dans ce cas-ci il n'y en a qu'une seule (sinon spécifier)

print(layer.GetExtent()) #affiche de l'emprise de la couche vectorielle

listvrt = []
i=0

for feat in layer: #une boucle sur les polygones dans la couche vectorielle
    if i <= 10 : 
        polygone = feat.GetGeometryRef().ExportToJson() #extraire les données géométriques et exporter en format JSON
        list_coord = extract_coord(polygone) #extraction des coordonnées des polygones
        print(list_coord)
        poly_id = feat.GetField("id") #extraire le id de l'object
        lulc = feat.GetField("lulc") #extraire le lulc (si slum ou pas slum) de l'object
        name_file='s2_b02_'+str(poly_id)+'_'+str(lulc)+'.tif' #création du nom du fichier de sortie
        filename = 'Images/s2_small_cut/B02/'+str(name_file)
        gdal.Warp(str(filename),rast, outputBounds = list_coord) #division en petite tuile correspondante aux polygone dans le shpfile
        listvrt.append(filename)
        i +=1
    
              


(259690.0, 266590.0, 9853680.0, 9860280.0)
(259690.0, 9860280.0, 259790.0, 9860180.0)
(259690.0, 9860180.0, 259790.0, 9860080.0)
(259690.0, 9860080.0, 259790.0, 9859980.0)
(259690.0, 9859980.0, 259790.0, 9859880.0)
(259690.0, 9859880.0, 259790.0, 9859780.0)
(259690.0, 9859780.0, 259790.0, 9859680.0)
(259690.0, 9859680.0, 259790.0, 9859580.0)
(259690.0, 9859580.0, 259790.0, 9859480.0)
(259690.0, 9859480.0, 259790.0, 9859380.0)
(259690.0, 9859380.0, 259790.0, 9859280.0)
(259690.0, 9859280.0, 259790.0, 9859180.0)


In [111]:
rast = gdal.Open('Images/s2_small_cut/B02/s2_b02_52739_0.tif')
band = rast.ReadAsArray()
print(band)

[[ -3.1803706  -4.1433983  -6.6385584  -9.118368  -10.022296   -8.631668
   -6.4746575  -5.917351   -7.022896   -6.3585486]
 [ -6.231778   -6.60423    -7.513828   -7.9901505  -7.6136155  -6.091342
   -4.8407702  -5.130327   -6.979833   -7.684854 ]
 [ -7.7216554  -7.895165   -8.70667    -7.2578697  -5.5923777  -4.1218195
   -3.5167878  -3.9842045  -5.515399   -6.7865934]
 [ -8.446246   -8.452781   -9.358833   -6.7285504  -4.3277855  -3.0610738
   -2.7796342  -3.1016464  -4.016515   -5.533632 ]
 [ -8.436233   -8.160825   -8.857988   -6.4021754  -4.305549   -3.4454105
   -3.0018084  -2.8723052  -3.0874527  -4.609898 ]
 [ -8.111731   -7.560599   -7.416033   -6.3661227  -5.5622582  -5.321836
   -4.8544064  -4.477586   -4.23302    -4.783431 ]
 [ -8.610285   -7.6678967  -6.7834272  -6.9550624  -7.29164    -7.6300206
   -7.292251   -6.8599706  -6.370966   -5.985206 ]
 [-10.249941   -8.762363   -7.3769345  -8.323401   -9.436078  -10.264026
  -10.23629    -9.968861   -9.480829   -8.367252 ]
 [-1